# IMPORTS

In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from typing import Optional

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from metrics import compute_single_col_score, get_tresholds, compute_weird_pred_proba_score
from sklearn.metrics import recall_score
from helper import make_prediction, check_path
from sklearn.model_selection import StratifiedKFold, train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2022-07-06 06:42:58.803124: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 06:42:58.836635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 06:42:58.836950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# MODEL TRAINING

In [5]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
test = pd.read_pickle(cfg.PREPARED_TEST_DATA_PATH)

In [6]:
train[cfg.CAT_UNORDERED_COLS] = train[cfg.CAT_UNORDERED_COLS].astype('string')
test[cfg.CAT_UNORDERED_COLS] = test[cfg.CAT_UNORDERED_COLS].astype('string')

In [7]:
train[cfg.REAL_COLS] = train[cfg.REAL_COLS].fillna(-1)
test[cfg.REAL_COLS] = test[cfg.REAL_COLS].fillna(-1)

In [8]:
X_train, Y_train = train.drop(cfg.TARGETS, axis=1), train[cfg.TARGETS]

In [9]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(cfg.TARGETS))), index=train.index, columns=cfg.TARGETS)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(cfg.TARGETS))), index=test.index, columns=cfg.TARGETS)
metrics = {}

In [10]:
EXPERIMENT_FAMILY_NAME = 'keras'
EXPERIMENT_NAME = 'baseline'
RANDOM_STATE = 77
N_SPLITS = 5
N_RANDOM_SEEDS = 1

In [11]:
CAT_COLS = cfg.CAT_UNORDERED_COLS

In [12]:
def df_to_dataset(data: pd.DataFrame, labels: Optional[pd.DataFrame]=None, shuffle: bool=True, batch_size: int=32):
  df = data.copy()
  df = {key: value.values[:, tf.newaxis] for key, value in data.items()}
  
  if labels is None:
    ds = tf.data.Dataset.from_tensor_slices((dict(df),))
  else:
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [13]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [14]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))


In [15]:
BATCH_SIZE = 16

In [16]:
from gc import callbacks
from telnetlib import X3PAD


cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)
test_ds = df_to_dataset(test, shuffle=False, batch_size=BATCH_SIZE)


fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, Y_train), total=N_SPLITS):

    train_ds = df_to_dataset(
        data=X_train.iloc[train_idx], 
        labels=Y_train.iloc[train_idx],
        batch_size=BATCH_SIZE)

    val_ds = df_to_dataset(
        data=X_train.iloc[val_idx],
        labels=Y_train.iloc[val_idx],
        shuffle=False, 
        batch_size=BATCH_SIZE)

    
    all_inputs = []
    encoded_features = []

    # Numerical features.
    for header in cfg.REAL_COLS:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)

    for header in cfg.CAT_ORDERED_COLS + cfg.BINARY_COLS:
        ordered_cat_col = tf.keras.Input(shape=(1,), name=header, dtype='int32')

        encoding_layer = get_category_encoding_layer(name=header,
                                                    dataset=train_ds,
                                                    dtype='int64',
                                                    max_tokens=None)
        encoded_ordered_cat_col = encoding_layer(ordered_cat_col)
        all_inputs.append(ordered_cat_col)
        encoded_features.append(encoded_ordered_cat_col)

    for header in cfg.CAT_UNORDERED_COLS:
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
        encoding_layer = get_category_encoding_layer(name=header,
                                                    dataset=train_ds,
                                                    dtype='string',
                                                    max_tokens=5)
        encoded_categorical_col = encoding_layer(categorical_col)
        all_inputs.append(categorical_col)
        encoded_features.append(encoded_categorical_col)


    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(256)(all_features)
    x = tf.keras.layers.LeakyReLU(alpha=0.15)(x)
    x = tf.keras.layers.Dropout(0.6)(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.15)(x)
    x = tf.keras.layers.Dropout(0.6)(x)
    
    output = tf.keras.layers.Dense(len(cfg.TARGETS))(x)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    # reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)


    model = tf.keras.Model(all_inputs, output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.0007,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=False,
            name='Adam'
            )
,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        # metrics=['AUC']
        )

        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):
        tf.random.set_seed(random_seed)

        model.fit(train_ds, epochs=100, validation_data=val_ds, verbose=1, callbacks=[early_stopping]) # reduce_lr

        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        model.save(os.path.join(model_path, model_name))
        
        pred_proba_oof.iloc[val_idx, :] += tf.nn.sigmoid(model.predict(val_ds)).numpy().squeeze()
        pred_proba_test.iloc[:, :] += tf.nn.sigmoid(model.predict(test_ds)).numpy().squeeze()
        gc.collect()
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (N_SPLITS * N_RANDOM_SEEDS)

2022-07-06 06:43:02.432260: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-06 06:43:02.432867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 06:43:02.433234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 06:43:02.433484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
48/48 [==============================] - 5s 43ms/step - loss: 0.4954 - val_loss: 0.3644
Epoch 2/100
48/48 [==============================] - 1s 16ms/step - loss: 0.4125 - val_loss: 0.3534
Epoch 3/100
48/48 [==============================] - 1s 16ms/step - loss: 0.3821 - val_loss: 0.3474
Epoch 4/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3667 - val_loss: 0.3447
Epoch 5/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3532 - val_loss: 0.3403
Epoch 6/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3375 - val_loss: 0.3363
Epoch 7/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3441 - val_loss: 0.3347
Epoch 8/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3460 - val_loss: 0.3364
Epoch 9/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3364 - val_loss: 0.3372
Epoch 10/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3337 - val_loss: 0.3384

INFO:tensorflow:Assets written to: /home/as/my_repositories/yar_digital_breakthrough_2022/checkpoints/keras/baseline/baseline_fold_0_rs_0/assets


...

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
48/48 [==============================] - 4s 39ms/step - loss: 0.4951 - val_loss: 0.3619
Epoch 2/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3992 - val_loss: 0.3490
Epoch 3/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3797 - val_loss: 0.3411
Epoch 4/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3745 - val_loss: 0.3388
Epoch 5/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3690 - val_loss: 0.3321
Epoch 6/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3548 - val_loss: 0.3314
Epoch 7/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3485 - val_loss: 0.3299
Epoch 8/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3487 - val_loss: 0.3321
Epoch 9/100
48/48 [==============================] - 1s 15ms/step - loss: 0.3391 - val_loss: 0.3312
Epoch 10/100
48/48 [==============================] - 1s 14ms/step - loss: 0.3370 - val_loss: 0.3319

INFO:tensorflow:Assets written to: /home/as/my_repositories/yar_digital_breakthrough_2022/checkpoints/keras/baseline/baseline_fold_4_rs_0/assets


INFO:tensorflow:Assets written to: /home/as/my_repositories/yar_digital_breakthrough_2022/checkpoints/keras/baseline/baseline_fold_4_rs_0/assets


40/40 [==============================] - 1s 10ms/step


In [17]:
# tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)


# PREDICT AND SAVE PREDICTIONS

In [18]:
tresholds = get_tresholds(train[cfg.TARGETS], pred_proba_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(pred_proba_test, tresholds, sample_submission)

[0.7066901920838184, 0.7017318401566611, 0.6047849706960687, 0.6905721300101404, 0.7248165311390486]
0.6857191328171475 0.04194922849980627


In [19]:
## BEST PARAMS
# [0.7066901920838184, 0.7017318401566611, 0.6047849706960687, 0.6905721300101404, 0.7248165311390486]
# 0.6857191328171475 0.04194922849980627

In [20]:
submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))